### Warning: Do not use ###
This notebook should not be used as the MAXAR Turkey Islahiye dataset is no longer used in the project due to the large number of incorrect labels.

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import h5py

import shutil
from keras.models import load_model
from keras import layers
from keras import models
from keras import optimizers

In [3]:
model1 = load_model('cc_model_with_dropout_4.h5')
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 sequential (Sequential)     (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                  

In [68]:
# test_ds = tf.data.Dataset.load('turkey/test')
test_ds = tf.data.Dataset.load('xBD_Dataset/xBD_test_2')

history = model1.evaluate(test_ds)

74/74 [==============================] - 1s 6ms/step - loss: 0.6368 - precision_1: 0.2375 - recall_1: 0.8963 - accuracy: 0.7922


In [5]:
print(history)

[4.472379207611084, 0.14612868428230286, 0.3529411852359772, 0.7033833265304565]


In [13]:
test_predictions = model1.predict(test_ds, batch_size = 32)

326/326 [==============================] - 2s 6ms/step


In [52]:
model1 = load_model('cc_model_with_dropout_2.h5')
# Disassemble layers
layers = [l for l in model1.layers]


# Stack everything back

x = layers[0].output
for i in range(2, len(layers)):
    if i < len(layers)-1:
        layers[i].trainable = False
    else:
        layers[i].trainable = True
    x = layers[i](x)

# Final touch
new_model = tf.keras.Model(inputs=layers[0].input, outputs=x)
new_model.summary()
    
new_model.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                 metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'accuracy'])



Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1_input (InputLay  [(None, 100, 100, 3)]    0         
 er)                                                             
                                                                 
 rescaling_1 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 64)       0  

In [7]:
# train_ds = tf.data.Dataset.load('turkey/train')
# val_ds = tf.data.Dataset.load('turkey/val')
train_ds = tf.data.Dataset.load('xBD_Dataset/xBD_train_2')
val_ds = tf.data.Dataset.load('xBD_Dataset/xBD_val_2')
test_ds = tf.data.Dataset.load('xBD_Dataset/xBD_test_2')

In [9]:
len(list(test_ds[0]))

TypeError: '_LoadDataset' object is not subscriptable

In [54]:
history1 = new_model.fit(
            train_ds,
            epochs=100,
            validation_data=val_ds,
#             class_weight={0:1,1:3}
            )

Epoch 1/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0021 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.1343 - val_precision_17: 0.8778 - val_recall_17: 0.9405 - val_accuracy: 0.9024
Epoch 2/100
6/6 [==============================] - 0s 20ms/step - loss: 2.9199e-04 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.0578 - val_precision_17: 0.9186 - val_recall_17: 0.9405 - val_accuracy: 0.9268
Epoch 3/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0013 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.0334 - val_precision_17: 0.9070 - val_recall_17: 0.9286 - val_accuracy: 0.9146
Epoch 4/100
6/6 [==============================] - 0s 19ms/step - loss: 2.5479e-04 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.0303 - val_precision_17: 0.9070 - val_recall_17: 0.9286 - val_accuracy: 0.9146
Epoch 5/100
6/6 [==============================] - 0s 18

6/6 [==============================] - 0s 19ms/step - loss: 1.4616e-05 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.4435 - val_precision_17: 0.8778 - val_recall_17: 0.9405 - val_accuracy: 0.9024
Epoch 70/100
6/6 [==============================] - 0s 20ms/step - loss: 8.8360e-06 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.4446 - val_precision_17: 0.8778 - val_recall_17: 0.9405 - val_accuracy: 0.9024
Epoch 71/100
6/6 [==============================] - 0s 19ms/step - loss: 8.3498e-06 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.4450 - val_precision_17: 0.8778 - val_recall_17: 0.9405 - val_accuracy: 0.9024
Epoch 72/100
6/6 [==============================] - 0s 19ms/step - loss: 6.4246e-05 - precision_17: 1.0000 - recall_17: 1.0000 - accuracy: 1.0000 - val_loss: 1.4508 - val_precision_17: 0.8778 - val_recall_17: 0.9405 - val_accuracy: 0.9024
Epoch 73/100
6/6 [==============================] - 0s 19

In [46]:
history = new_model.evaluate(test_ds)

74/74 [==============================] - 1s 6ms/step - loss: 0.3027 - precision_14: 0.4118 - recall_14: 0.8110 - accuracy: 0.9061


In [11]:
counter = 0
for element in test_ds:
    preds = model1.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'turkey/test_predictions/undamaged/'
        else:
            dstatus = 'turkey/test_predictions/damaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 65ms/step


In [ ]:
counter

In [56]:
counter = 0
for element in test_ds:
    preds = new_model.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'xBD_Dataset/test_predictions/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_predictions/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 19ms/step


In [13]:
counter

2353

In [69]:
counter = 0
dataset = test_ds.enumerate()
arr = []
for element in dataset.as_numpy_iterator():
    for i in range(len(element[1][1])):
        dmg = element[1][1][i]   
        img = PIL.Image.fromarray(element[1][0][i].astype(np.uint8))
        if dmg == 1:
            dstatus = 'xBD_Dataset/test_truth/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_truth/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')